# 第 6 章 會寫程式與生圖的內建工具

## 前置工作

請先執行以下這幾個儲存格，便於後續測試：

In [ ]:
from IPython.display import Markdown, display
from google.colab import userdata
from rich.pretty import pprint
from pydantic import BaseModel, Field, ConfigDict
from io import BytesIO
import requests
import openai
import sys
import os
import json

In [ ]:
client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
def upload_file(file_path):
    try:
        if (file_path.startswith('http://')
            or file_path.startswith('https://')):
            response = requests.get(file_path)
            filename = response.headers.get(
                'content-disposition',
                None
            )
            if filename:
                filename = filename.split('filename=')[-1]
                filename = filename.strip('\'"')
            else:
                filename = file_path.split('/')[-1]
            response = client.files.create(
                file=(filename, response.content),
                purpose='user_data'
            )
        else:
            with open(file_path, 'rb') as file:
                response = client.files.create(
                    file=file,
                    purpose='user_data'
                )
    except:
        return None
    return response.id

In [ ]:
class BaseComand:
    def __init__(self, command, tool_name, icon, verbose=False):
        self.command = command      # 指令
        self.tool_name = tool_name  # 工具名稱
        self.icon = icon            # 工具圖示字元
        self.verbose = verbose      # 啟用詳細輸出
        self.extra_args = {}        # 要額外送給模型的參數

    def handle_command(self, chat, cmd):
        # 不是正確的指令開頭（空字串會是 True）
        if not cmd.startswith(self.command):
            return False
        return True

    def handle_event(self, chat, stream, event):
        return None # 預設不處理交給下一個指令處理器

In [ ]:
from urllib.parse import unquote

class WebSearchCommand(BaseComand):
    def __init__(self, verbose=False):
        super().__init__(
            '/w',
            'web_search_preview',
            '🌐',
            verbose
        )

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        idx = chat.find_tool_index(self.tool_name)
        if idx == -1:
            chat.tools.append({
                'type': self.tool_name
            })
        else:
            chat.tools.pop(idx)
        return True

    def show_search_results(self, response):
        if response.output[0].type != "web_search_call":
            return
        content = response.output[1].content[0]
        for i, annotaion in enumerate(
            content.annotations, start=1
        ):
            print(f'{i}. {annotaion.title}')
            print(f'   {unquote(annotaion.url)}')

    def handle_event(self, chat, stream, event):
        if not self.verbose: return None
        if event.type == 'response.completed':
            self.show_search_results(event.response)
        return None

In [ ]:
class FileSearchCommand(BaseComand):
    def __init__(self, vector_store_id=None, verbose=False):
        super().__init__(
            '/f',
            'file_search',
            '🔍',
            verbose
        )
        self.vector_store_id = vector_store_id

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        idx = chat.find_tool_index(self.tool_name)
        if len(cmd) < 4: # /f[.]，不是冒號加檔名/網址
            if self.vector_store_id == None:
                print('請先使用 /f:[路徑|網址]上傳檔案')
                return True # 沒有向量資料庫無法切換
            turn_on = (idx == -1) # 切換開/關檔案檢索
        else: # /f:檔名|網址，上傳檔案並開啟檢索功能
            turn_on = True
            file_path = cmd[3:]
            file_id= upload_file(file_path)
            if not file_id:
                print(f'無法上傳檔案：{file_path}')
                return True
            if self.vector_store_id == None:
                vector_store = client.vector_stores.create(
                    name="temp",
                    file_ids=[file_id],
                )
                self.vector_store_id = vector_store.id
            else:
                client.vector_stores.files.create(
                    self.vector_store_id,
                    file_id=file_id
                )
        if turn_on:
            chat.tools.append({
                'type': self.tool_name,
                'vector_store_ids': [self.vector_store_id]
            })
            self.extra_args = {
                'include': ['file_search_call.results']
            }
        else:
            chat.tools.pop(idx)
            self.extra_args = {}
        return True

    def remove_vector_store(self, chat):
        if self.vector_store_id == None: return
        idx = chat.find_tool_index('file_search')
        if idx: chat.tools.pop(idx)
        response = client.vector_stores.files.list(
            self.vector_store_id
        )
        for vector_file in response.data:
            client.files.delete(vector_file.id)
        client.vector_stores.delete(self.vector_store_id)

    def show_file_search_results(self, response):
        if response.output[0].type != 'file_search_call':
            return
        results = response.output[0].results
        if not results: return
        for i, result in enumerate(results, start=1):
            display(Markdown('---'))
            print(f'{i}. {result.filename}({result.score})')
            display(Markdown('---'))
            display(Markdown(result.text))

    def handle_event(self, chat, stream, event):
        if not self.verbose: return None
        if event.type == 'response.completed':
            self.show_file_search_results(event.response)
        return None

In [ ]:
class FunctionCallingCommand(BaseComand):
    def __init__(self, tools=None, verbose=False):
        super().__init__(
            '/t',
            'function',
            '',
            verbose
        )
        self.tools = tools or []
        self.enabled = False

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        if not self.enabled: # 加入自訂函式工具
            chat.tools.extend(self.tools)
        else: # 移除自訂工具函式
            chat.tools = [
                tool for tool in chat.tools
                if tool['type'] != self.tool_name
            ]
        self.enabled = not self.enabled
        return True

    # 叫用單一函式並且將函式執行結果組成訊息後傳回
    def make_tool_msg(self, tool_call):
        tool_info = f'{tool_call.name}(**{tool_call.arguments})'
        if self.verbose: print(f'叫用：{tool_info}')
        func = eval(tool_call.name)
        args = json.loads(tool_call.arguments)
        result = func(**args)
        return {   # 建立可傳回函式執行結果的字典
            "type": "function_call_output", # 以工具角色送出回覆
            "call_id": tool_call.call_id, # 叫用函式的識別碼
            "output": result # 函式傳回值
        }

    def call_tools(self, tool_calls):
        msgs = []
        for tool_call in tool_calls:
            if tool_call.type == 'function_call':
                msgs.append(self.make_tool_msg(tool_call))
        return msgs if msgs else None

    def handle_event(self, chat, stream, event):
        if not self.enabled: return None
        if event.type != 'response.completed':
            return None
        # 呼叫函式
        tool_calls = event.response.output
        tool_results = self.call_tools(tool_calls)
        if tool_results: return tool_calls + tool_results
        return None

In [ ]:
import subprocess

def shell_helper(comment, shell_command):
    # 啟動子行程
    process = subprocess.Popen(
        shell_command,
        shell=True,             # 在 shell 中執行
        stdout=subprocess.PIPE, # 擷取標準輸出
        stderr=subprocess.PIPE, # 擷取錯誤輸出
        text=True               # 以文字形式返回
    )

    result = '執行結果：\n\n```\n'

    # 即時讀取輸出
    while True:
        output = process.stdout.readline()
        # 如果沒有輸出且行程結束
        if output == '' and process.poll() is not None:
            break
        if output:
            result += output

    result += "```"

    # 檢查錯誤輸出
    error = process.stderr.read()
    if error:
        result += f"\n\n錯誤: {error}"

    # 等待行程結束並取得返回碼
    return_code = process.wait()
    result += f"\n\n命令執行完成，返回碼: {return_code}\n\n"
    return result

In [ ]:
class ShellHelper(BaseModel):
    comment: str = Field(
        description='判斷要執行指定的 shell 指令的原因'
    )
    shell_command: str = Field(
        description='要執行的 shell 指令'
    )

In [ ]:
shell_helper_tool = {
    'type': 'function',
    "name": "shell_helper",
    "description": "我可以執行 shell 指令操控電腦",
    "parameters": ShellHelper.model_json_schema(),
}

## 6-1 使用內建工具執行 shell 指令

In [ ]:
response = client.responses.create(
    model='codex-mini-latest',
    tools=[{'type': 'local_shell'}],
    instructions='使用繁體中文',
    input= '我在哪個路徑下'
)

pprint(response)

In [ ]:
for output in response.output:
    if output.type != 'local_shell_call':
        continue
    action = output.action
    call_id = output.call_id

In [ ]:
import subprocess

result = subprocess.run(
    action.command,
    capture_output=True
)

In [ ]:
output_md = (
    f'執行結果\n\n```\n'
    f'{result.stdout.decode("utf8")}\n'
    f'```\n\n錯誤訊息\n\n```\n'
    f'{result.stderr.decode("utf8")}\n'
    f'```\n\n結束碼：{result.returncode}'
)
print(output_md)

In [ ]:
response = client.responses.create(
    model='codex-mini-latest',
    tools=[{'type': 'local_shell'}],
    instructions='使用繁體中文',
    input= [{
        "type": "local_shell_call_output",
        "call_id": call_id,
        "output": output_md
    }],
    previous_response_id=response.id
)

print(response.output_text)

## 6-2 Code Interpreter 內建工具

- [code interpreter 文件](https://platform.openai.com/docs/guides/tools-code-interpreter)

自動建立容器在 Jupyter 中執行程式：

- 容器會在未使用後 20 分鐘失效，容器內的所有資料都會消失。
- 不同次使用 code interpreter 工具可能會沿用既有的容器。
- code interpreter 的計費就是每開一個容器 0.03 美金。

In [ ]:
response = client.responses.create(
    model='gpt-4.1-nano',
    tools=[{
        'type': 'code_interpreter',
        'container': {'type': 'auto'}
    }],
    instructions=(
        '你是使用繁體中文的助理，當遇到數學相關問題時，'
        '不要先回答，一律寫程式再回覆'
    ),
    input='3.8 和 3.11 哪一個大？'
)

print(response.output_text)

In [ ]:
pprint(response)

In [ ]:
print(response.output[0].code)

### 取得程式碼輸出結果

In [ ]:
response = client.responses.create(
    model='gpt-4.1-nano',
    tools=[{
        'type': 'code_interpreter',
        'container': {'type': 'auto'}
    }],
    instructions=(
        '你是使用繁體中文的助理，當遇到數學相關問題時，'
        '不要先回答，一律寫程式再回覆'
    ),
    input='3.8 和 3.11 哪一個大？',
    include=['code_interpreter_call.outputs']
)

pprint(response)

### 使用串流方式

In [ ]:
response = client.responses.create(
    model='gpt-4.1-nano',
    tools=[{
        'type': 'code_interpreter',
        'container': {'type': 'auto'}
    }],
    instructions=(
        '你是使用繁體中文的助理，當遇到數學相關問題時，'
        '不要先回答，一律寫程式再回覆'
    ),
    input='3.8 和 3.11 哪一個大？',
    include=['code_interpreter_call.outputs'],
    stream=True
)

for event in response:
    pprint(event)

### 由 Code Interpreter 處理檔案

In [ ]:
!curl https://flagtech.github.io/images/tomica.jpg -o tomica.jpg


In [ ]:
file_id = upload_file(
    'https://flagtech.github.io/images/tomica.jpg'
)

複雜的問題可能會跑超多輪工具跑很久，甚至逾時：

In [ ]:
response = client.responses.create(
    model='gpt-4.1-mini',
    tools=[{
        'type': 'code_interpreter',
        'container': {
            'type': 'auto',
            'file_ids': [file_id]
        }
    }],
    instructions='對於理工數學影像處理問題，'
                 '用 the python tool 寫程式處理',
    input='幫我把照片變成灰階'
    # input='將照片去背'
)

print(response.output_text)

In [ ]:
pprint(response)

目前以下只會傳回 None：

```python
annotation = response.output[-1].content[0].annotations[0]
file_response = client.containers.files.content.retrieve(
    container_id=annotation.container_id,
    file_id=annotation.file_id
)

pprint(file_response)
```

所以改用原始的 REST API：

In [ ]:
def get_file_content_from_container(container_id, file_id):
    url = (
        'https://api.openai.com/v1/containers/'
        f'{container_id}'
        '/files/'
        f'{file_id}'
        '/content'
    )
    response = requests.get(
        url,
        headers={
            'Authorization': f'Bearer {client.api_key}'
        }
    )
    if response.status_code != 200:
        return None
    return response.content

In [ ]:
def get_code_interpreter_files(response):
    files = []
    for output in response.output:
        if output.type != 'message':
            continue
        for content in output.content:
            for annotation in content.annotations:
                if annotation.type != 'container_file_citation':
                    continue
                files.append((
                    annotation.container_id,
                    annotation.file_id,
                    annotation.filename
                ))
                content = get_file_content_from_container(
                    file_id=annotation.file_id,
                    container_id=annotation.container_id,
                )
                if not content:
                    print(f'無法下載 {annotation.filename}')
                    continue
                with open(annotation.filename, 'wb') as f:
                    f.write(content)
    return files

In [ ]:
files = get_code_interpreter_files(response)
pprint(files)

其實容器自己會失效，資料也會消失，並沒有真的需要刪除檔案以及容器的必要。

In [ ]:
for file in files:
    client.containers.files.delete(
        container_id=file[0],
        file_id=file[1]
    )

In [ ]:
client.containers.delete(files[0][0])

補充說明，刪除容器中檔案與容器本身的 REST API 版本：

```python
def delete_container_file(container_id, file_id):
    url = (
        'https://api.openai.com/v1/containers/'
        f'{container_id}'
        '/files/'
        f'{file_id}'
    )
    response = requests.delete(
        url,
        headers={
            'Authorization': f'Bearer {client.api_key}'
        }
    )
    return response.status_code == 200
```

```python
def delete_container(container_id):
    url = (
        'https://api.openai.com/v1/containers/'
        f'{container_id}'
    )
    response = requests.delete(
        url,
        headers={
            'Authorization': f'Bearer {client.api_key}'
        }
    )
    return response.status_code == 200
```

```python
for file in files:
    if delete_container_file(file[0], file[1]):
        print(f'成功刪除 {file[2]}')
```

### 手動建立容器

你也可以自己建立容器，並指定使用該容器，這也可以指定使用自動建立的容器。容器中的檔案可以在建立時同時加入，也可以在建立後加入。

In [ ]:
response = client.containers.create(
    name='test',
    # expires_after={
    #     'anchor': 'last_active_at',
    #     'minutes': 20
    # }
)

pprint(response)

REST API 版本：

```python
def create_container(name, expires_after=20, file_ids=None):
    url = 'https://api.openai.com/v1/containers'
    data = {
        'name': name,
        'expires_after': {
            'anchor': 'last_active_at',
            'minutes': expires_after,
        },
        'file_ids': file_ids if file_ids else []
    }
    response = requests.post(
        url,
        headers={
            'Authorization': f'Bearer {client.api_key}'
        },
        json=data
    )
    if response.status_code != 200:
        return None
    return response.json()['id']
```

In [ ]:
response = client.containers.files.create(
    container_id=response.id,
    file_id=file_id
)

pprint(respsone)

REST API 版本：

```python
def add_file_to_container(container_id, file_id):
    url = (
        'https://api.openai.com/v1/containers'
        f'/{container_id}/files'
    )
    response = requests.post(
        url,
        headers={
            'Authorization': f'Bearer {client.api_key}'
        },
        json={
            'file_id': file_id
        }
    )
    if response.status_code != 200:
        return None
    return response.json()['id']
```

In [ ]:
response = client.responses.create(
    model='gpt-4.1-mini',
    tools=[{
        'type': 'code_interpreter',
        'container': response.container_id
    }],
    instructions='對於理工數學影像處理問題，'
                 '用 the python tool 寫程式處理',
    input='將照片變成藍色調色階效果的圖'
)

In [ ]:
pprint(response)

In [ ]:
files = get_code_interpreter_files(response)
print(files[-1][-1])

## 6-3 內建生圖工具

- 背後生圖的是 [gpt-image-1](https://platform.openai.com/docs/models/gpt-image-1) 模型
- [可用選項](https://platform.openai.com/docs/guides/image-generation#customize-image-output)
- [費用](https://platform.openai.com/docs/guides/image-generation#cost-and-latency)

In [ ]:
response = client.responses.create(
    model='gpt-4.1-mini',
    input='用手塚治虫風格生成工程師在海灘上寫程式的圖',
    tools=[{
        'type': 'image_generation',
        'size': 'auto',
        'quality': 'auto',
    }]
)

pprint(response)

In [ ]:
import base64
from IPython.display import Image

def get_img_b64(outputs):
    img_data = [
        output.result for output in outputs
        if output.type == 'image_generation_call'
    ]
    return img_data[0] if img_data else None

def get_img_obj(img_b64, width=300):
    return Image(base64.b64decode(img_b64), width=width)

def save_img(img_b64, filename='gen.png'):
    with open(filename, 'wb') as f:
        f.write(base64.b64decode(img_b64))

In [ ]:
img_b64 = get_img_b64(response.output)
display(get_img_obj(img_b64))
save_img(img_b64)

In [ ]:
print(response.output_text)

### 串接回應持續修改

In [ ]:
response = client.responses.create(
    model='gpt-4.1-mini',
    input='把它變成《攻殼機動隊》的風格，但畫面要細緻明亮',
    tools=[{
        'type': 'image_generation',
    }],
    previous_response_id=response.id
)

img_b64 = get_img_b64(response.output)
display(get_img_obj(img_b64))

### 以串流模式取得過程變化圖

會有 partial_images+1 次的 output，最後一次就是完整的結果

In [ ]:
response = client.responses.create(
    model='gpt-4.1-mini',
    input='用吉卜力風格生成工程師在海灘上寫程式的圖',
    tools=[{
        'type': 'image_generation',
        'partial_images': 3 # 串流模式才能用
    }],
    stream=True
)

for output in response:
    pprint(output)

In [ ]:
response = client.responses.create(
    model='gpt-4.1-mini',
    input='用蒙德里安風格生成工程師在海灘上寫程式的圖',
    tools=[{
        'type': 'image_generation',
        'partial_images': 3 # 串流模式才能用
    }],
    stream=True
)

imgs = []
display_handle = display(None, display_id=True)

for event in response:
    if event.type == 'response.completed':
        b64_data = get_img_b64(event.response.output)
    elif event.type == (
        'response.image_generation_call.partial_image'
    ):
        b64_data = event.partial_image_b64
    else: continue
    display_handle.update(get_img_obj(b64_data))
    imgs.append(b64_data)

In [ ]:
imgs[-2] == imgs[-1]

### 在背景執行 Responses API

In [ ]:
response = client.responses.create(
    model='gpt-4.1-mini',
    input='用梵谷風格生成工程師在海灘上寫程式的圖',
    tools=[{
        'type': 'image_generation',
        'size': 'auto',
        'quality': 'auto',
    }],
    background=True
)

response = client.responses.retrieve(response.id)
pprint(response)

In [ ]:
import time

while response.status != 'completed':
    time.sleep(5)
    response = client.responses.retrieve(response.id)
    print(response.status)

In [ ]:
img_b64 = get_img_b64(response.output)
display(get_img_obj(img_b64))

## 6-4 幫聊天程式加入新內建工具

要特別留意有些圖形字元，像是 '⌨️'、'🖌️' 因為有加上了[**變體選擇符 (variation selector)**](https://grok.com/share/bGVnYWN5_78048bb7-7098-4fc2-818f-7475e3188243)，所以會被計為 2 個字元，在底下計算自訂工具個數時會出錯。

### 修改 Chat 類別

In [ ]:
class Chat:
    def __init__(self, client, **kwargs):
        self._client = client
        self._last_id = kwargs.pop('last_id', None)
        # 限制工具執行圈數，避免無窮盡叫用工具
        self._max_tools_rounds = kwargs.pop('max_tools_rounds', 4)
        self.tools = [] # 預設沒有使用工具
        self._commands = kwargs.pop('commands', [])

    def get_reply_text(self, msg, **kwargs) -> str:
        instructions = kwargs.pop('instructions', '使用繁體中文')
        model = kwargs.pop('model', 'gpt-4.1-nano')
        stream = kwargs.pop('stream', False)
        tool_results = [] # 函式叫用的相關資訊
        for command in self._commands:
            extra_args = {
                k: kwargs[k] + command.extra_args[k]
                if k in kwargs
                else command.extra_args[k]
                for k in command.extra_args
            }
            kwargs.update(extra_args)
        try:
            messages = [{'role': 'user', 'content': msg}]
            for _ in range(self._max_tools_rounds):
                # 方便稍後串接函式叫用資訊
                if tool_results: # 串接叫用函式的資訊
                    messages += tool_results
                response = self._client.responses.create(
                    instructions=instructions,
                    model=model,
                    input=messages,
                    stream=True, # 都以串流方式處理，簡化程式邏輯
                    previous_response_id=self._last_id, # 串接回應
                    **kwargs
                )
                for event in response:
                    for command in self._commands:
                        result = command.handle_event(
                            self, stream, event
                        )
                        if result: break
                    tool_results = []
                    if isinstance(result, list):
                        # 工具要送回給模型的訊息串列
                        tool_results = result
                        break
                    elif result: # 指令處理器產生的內容
                        yield result
                    if event.type == 'response.output_text.delta':
                        if stream: yield event.delta
                    elif event.type == (
                        'response.output_text.done'
                    ):
                        # 非串流模式要傳回完整內容
                        if not stream:
                            yield event.text
                    elif event.type == 'response.completed':
                        # 更新最後回應的識別碼
                        self._last_id = event.response.id
                if not tool_results:
                    # 沒有要送回給模型的訊息
                    # 表示已經成功生成內容
                    break
        except openai.APIError as err:
            print(f'Error:{err.body["message"]}', file=sys.stderr)
            return ''

    def find_tool_index(self, tool_type):
        for i, tool in enumerate(self.tools):
            if tool['type'] == tool_type: return i
        return -1

    def _get_prompt(self):
        prompt = ''
        for command in self._commands:
            idx = self.find_tool_index(command.tool_name)
            if idx != -1:
                prompt += f'{command.icon}'
        user_tools_count = len(self.tools) - len(prompt)
        prompt += f'(🛠️{user_tools_count})>>> '
        return prompt

    def _process_command(self, cmd):
        for command in self._commands:
            if command.handle_command(self, cmd):
                return True
        return False

    def loop(self, **kwargs) -> None:
        print("直接按 ↵ 可結束對話")
        while True:
            user_msg = input(self._get_prompt())
            if not user_msg.strip(): break # 直接 ↵ 就結束
            if self._process_command(user_msg):
                continue # 指令不需回覆，回頭讓使用者輸入
            text = ''
            display_handle = display(text, display_id=True)
            display_extra_handle = None
            for reply in self.get_reply_text(
                user_msg,
                tools=self.tools, # 傳入要使用的工具
                **kwargs
            ):
                if not isinstance(reply, str):
                    if not display_extra_handle:
                        display_extra_handle = display(
                            reply, display_id=True
                        )
                    else:
                        display_extra_handle.update(reply)
                    continue
                text += reply
                if os.path.exists(text):
                    display_handle.update(Markdown(f' text'))
                else:
                    display_handle.update(Markdown(text))

    def save(self, filename) -> None:
        with open(filename, 'wb') as f:
            pickle.dump(
                {
                    'last_id': self._last_id,
                    'tools': self.tools,
                    'commands': self._commands
                },
                f
            )

    def load(self, filename) -> None:
        with open(filename, 'rb') as f:
            data = pickle.load(f)
            self._last_id = data['last_id']
            self.tools = data['tools']
            self._commands = data['commands']

    def show_thread(self):
        if not self._last_id: return
        inputs = client.responses.input_items.list(self._last_id)
        response = client.responses.retrieve(self._last_id)
        for item in inputs.data[::-1]:
            # 略過函式叫用指示等非訊息內容
            if item.type != 'message': continue
            prompt = ">>> " if item.role == 'user' else ''
            for content in item.content:
                print(f'{prompt}{content.text}')
        print(response.output_text)

    def delete_thread(self):
        if not self._last_id: return
        last_id = self._last_id
        while last_id:
            response = client.responses.retrieve(last_id)
            last_id, curr_id = (
                response.previous_response_id,
                last_id
            )
            client.responses.delete(curr_id)
        self._last_id = None

### 處理 Code Interpreter 指令的類別

In [ ]:
class CodeInterpreterCommand(BaseComand):
    def __init__(self, verbose=False):
        super().__init__(
            '/c',
            'code_interpreter',
            '🐍',
            verbose
        )

    def get_file_content_from_container(
            self, container_id, file_id
    ):
        url = (
            'https://api.openai.com/v1/containers/'
            f'{container_id}'
            '/files/'
            f'{file_id}'
            '/content'
        )
        response = requests.get(
            url,
            headers={
                'Authorization': f'Bearer {client.api_key}'
            }
        )
        if response.status_code != 200:
            return None
        return response.content

    def get_code_interpreter_files(self, response):
        files = []
        for output in response.output:
            if output.type != 'message':
                continue
            for content in output.content:
                for annotation in content.annotations:
                    if annotation.type != (
                        'container_file_citation'
                    ):
                        continue
                    files.append((
                        annotation.container_id,
                        annotation.file_id,
                        annotation.filename
                    ))
                    content = (
                        self.get_file_content_from_container(
                        annotation.container_id,
                        annotation.file_id
                    ))
                    if not content:
                        print(f'無法下載 {annotation.filename}')
                        continue
                    with open(annotation.filename, 'wb') as f:
                        f.write(content)
        return files

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        file_ids = []
        if len(cmd) > 3:
            file_id= upload_file(cmd[3:])
            if not file_id:
                print(f'無法上傳檔案：{file_path}')
                return True
            file_ids.append(file_id)
        idx = chat.find_tool_index(self.tool_name)
        if idx == -1:
            chat.tools.append({
                'type': self.tool_name,
                'container': {
                    'type': 'auto',
                    'file_ids': file_ids
                }
            })
            self.extra_args = {
                'include': ['code_interpreter_call.outputs'],
            }
        else:
            chat.tools.pop(idx)
            self.extra_args = {}
        return True

    def handle_event(self, chat, stream, event):
        if event.type == 'response.completed':
            files = self.get_code_interpreter_files(
                event.response
            )
            for f in files:
                print(f'已下載：{f[2]}')
            return None
        if not self.verbose: return None
        if event.type == (
            'response.code_interpreter_call.'
            'in_progress'
        ):
            if stream: return '\n```python\n'
        elif event.type == (
            'response.'
            'code_interpreter_call_code.delta'
        ):
            if stream: return event.delta
        elif event.type == (
            'response.'
            'code_interpreter_call_code.done'
        ):
            if stream: return '\n```\n\n'
            else: return f'\n\n```python\n{event.code}\n```\n\n'
        elif event.type == 'response.output_item.done':
            if event.item.type == 'code_interpreter_call':
                results = (
                    f'\n結果是：\n\n```\n'
                    f'{event.item.outputs[0]["logs"]}'
                    f'\n```\n\n'
                )
                return results
        return None

### 處理生圖指令的類別

In [ ]:
from IPython.display import Image
import base64

class ImageGenerationCommand(BaseComand):
    def __init__(self, verbose=False):
        super().__init__(
            '/i',
            'image_generation',
            '🎨',
            verbose=verbose,
        )

    def get_img_b64(self, outputs):
        img_data = [
            output.result for output in outputs
            if output.type == 'image_generation_call'
        ]
        return img_data[0] if img_data else None

    def get_img_obj(self, img_b64, width=300):
        return Image(base64.b64decode(img_b64), width=width)

    def save_img(self, img_b64, filename='gen.png'):
        with open(filename, 'wb') as f:
            f.write(base64.b64decode(img_b64))

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        idx = chat.find_tool_index(self.tool_name)
        if idx == -1:
            chat.tools.append({
                'type': self.tool_name,
                'partial_images': 3 if self.verbose else 1
            })
        else:
            chat.tools.pop(idx)
        return True

    def handle_event(self, chat, stream, event):
        if event.type == 'response.completed':
            b64_data = self.get_img_b64(event.response.output)
            if not b64_data: return None
        elif event.type == (
            'response.image_generation_call.partial_image'
        ):
            b64_data = event.partial_image_b64
        else: return None
        return self.get_img_obj(b64_data)

### 測試全功能的聊天程式

In [ ]:
file_search_command = FileSearchCommand(verbose=True)

chat = Chat(
    client,
    commands=[
        CodeInterpreterCommand(verbose=True),
        ImageGenerationCommand(verbose=True),
        FunctionCallingCommand(
            tools=[shell_helper_tool],
            verbose=True
        ),
        file_search_command,
        WebSearchCommand(verbose=True)
    ]
)

In [ ]:
chat.loop(
    model='gpt-4.1-mini',
    stream=True,
    instructions='使用繁體中文，'\
                 '對於數理問題都先寫程式執行後再回覆'
)

In [ ]:
file_search_command.remove_vector_store(chat)
chat.delete_thread()